In [1]:
%matplotlib auto

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib.widgets import LassoSelector
from matplotlib.path import Path

Using matplotlib backend: Qt5Agg


In [2]:
x = np.random.randint(1, 35, 10)
y = np.random.randint(1, 35, 10)
ax = plt.subplot()
ax.plot(x, y)

def onselect(verts):
    print(verts)

lasso = LassoSelector(ax, onselect)
plt.show()

[(2.8274193548387103, 9.366666666666665), (2.8274193548387103, 9.2), (2.72983870967742, 8.616666666666665), (2.6322580645161295, 7.699999999999999), (2.6322580645161295, 7.116666666666665), (2.5834677419354843, 6.866666666666665), (2.5834677419354843, 6.616666666666665), (2.5834677419354843, 6.199999999999999), (2.5834677419354843, 6.033333333333332), (2.5834677419354843, 5.949999999999999), (2.5834677419354843, 5.866666666666665), (2.6322580645161295, 5.699999999999999), (2.6810483870967747, 5.699999999999999), (2.6810483870967747, 5.616666666666665), (2.778629032258065, 5.449999999999999), (2.9250000000000007, 5.449999999999999), (2.973790322580646, 5.449999999999999), (3.0713709677419354, 5.449999999999999), (3.168951612903226, 5.449999999999999), (3.3153225806451614, 5.533333333333332), (3.3641129032258066, 5.616666666666665), (3.461693548387097, 5.866666666666665), (3.5592741935483874, 5.949999999999999), (3.6080645161290326, 6.116666666666665), (3.754435483870968, 6.2833333333333

In [3]:
class SelectFromCollection:
    def __init__(self, ax, collection, alpha_other=0.3):
        self.canvas = ax.figure.canvas
        self.collection = collection
        self.alpha_other = alpha_other

        self.xys = collection.get_offsets()
        self.Npts = len(self.xys)

        # Ensure that we have separate colors for each object
        self.fc = collection.get_facecolors()
        if len(self.fc) == 0:
            raise ValueError('Collection must have a facecolor')
        elif len(self.fc) == 1:
            self.fc = np.tile(self.fc, (self.Npts, 1))

        self.lasso = LassoSelector(ax, onselect=self.onselect)
        self.ind = []

    def onselect(self, verts):
        path = Path(verts)
        self.ind = np.nonzero(path.contains_points(self.xys))[0]
        self.fc[:, -1] = self.alpha_other
        self.fc[self.ind, -1] = 1
        self.collection.set_facecolors(self.fc)
        self.canvas.draw_idle()

    def disconnect(self):
        self.lasso.disconnect_events()
        self.fc[:, -1] = 1
        self.collection.set_facecolors(self.fc)
        self.canvas.draw_idle()


# Fixing random state for reproducibility
np.random.seed(19680801)

data = np.random.rand(100, 2)

subplot_kw = dict(xlim=(0, 1), ylim=(0, 1), autoscale_on=False)
fig, ax = plt.subplots(subplot_kw=subplot_kw)

pts = ax.scatter(data[:, 0], data[:, 1], s=80)
selector = SelectFromCollection(ax, pts)

def accept(event):
    if event.key == "enter":
        print("Selected points:")
        print(selector.xys[selector.ind])
        selector.disconnect()
        ax.set_title("")
        fig.canvas.draw()

fig.canvas.mpl_connect("key_press_event", accept)
ax.set_title("Press enter to accept selected points.")

plt.show()